# Imports necessários

In [51]:
import string
import pandas as pd
from unidecode import unidecode
from fuzzywuzzy import fuzz
import re
from random import randint
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import os

# Lista com os DF

In [52]:
list_data_frames = []

def inserir_df(df):
    list_data_frames.append(df)

In [53]:
df_despesas_2013 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2013/04_analise_tratamento_carga/dados_acumulados_2013.csv', sep=';')
inserir_df(df_despesas_2013)

df_despesas_2014 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2014/04_analise_tratamento_carga/dados_acumulados_2014.csv', sep=';')
inserir_df(df_despesas_2014)

df_despesas_2015 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2015/04_analise_tratamento_carga/dados_acumulados_2015.csv', sep=';')
inserir_df(df_despesas_2015)

df_despesas_2016 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2016/04_analise_tratamento_carga/dados_acumulados_2016.csv', sep=';')
inserir_df(df_despesas_2016)

df_despesas_2017 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2017/04_analise_tratamento_carga/dados_acumulados_2017.csv', sep=';')
inserir_df(df_despesas_2017)

df_despesas_2018 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2018/04_analise_tratamento_carga/dados_acumulados_2018.csv', sep=';')
inserir_df(df_despesas_2018)

df_despesas_2019 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2019/04_analise_tratamento_carga/dados_acumulados_2019.csv', sep=';')
inserir_df(df_despesas_2019)

df_despesas_2020 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2020/04_analise_tratamento_carga/dados_acumulados_2020.csv', sep=';')
inserir_df(df_despesas_2020)

df_despesas_2021 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2021/04_analise_tratamento_carga/dados_acumulados_2021.csv', sep=';')
inserir_df(df_despesas_2021)

df_despesas_2022 = pd.read_csv('C:/Users/allys/Dados_TCC/TCC2/dados/02_despesas/2022/04_analise_tratamento_carga/dados_acumulados_2022.csv', sep=';')
inserir_df(df_despesas_2022)

In [54]:
df_despesas_2013.head()

,Número,Data,Nome,CPF/CNPJ,Empenhado(R$),Liquidado(R$),Pago(R$),Saldo(R$),Tipo
0,5,03/01/2013,CAGEPA-COM DE AGUA E ESGOTOS DA PARAIBA,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,17,30/01/2013,Prefeitura Municipal de Cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,68,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,69,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,70,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


In [55]:
df_despesas_2022.head()

,DATA,CODIGO ORGAO,ORGAO,CODIGO DESPESA,DESPESA,DATA INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,29/12/2022,2140,Fundo Municipal de Assistência Social,339030,Material de Consumo,09/01/2023,9538.85,0.00,0.00,9538.85,1
1,29/12/2022,2140,Fundo Municipal de Assistência Social,339030,Material de Consumo,09/01/2023,8010.40,0.00,0.00,8010.40,1
2,29/12/2022,2040,Secretaria Municipal da Fazenda Pública,339039,Outros Serviços de Terceiros - Pessoa Jurídica,09/01/2023,166.36,166.36,166.36,0.00,1
3,29/12/2022,2090,Secretaria Municipal de Educação,449052,Equipamentos e Material Permanente,09/01/2023,63000.00,0.00,0.00,63000.00,1
4,29/12/2022,2090,Secretaria Municipal de Educação,339030,Material de Consumo,09/01/2023,3640.80,0.00,3640.80,0.00,1


# Padronizando os dataframes para junção

### Padronizando colunas

In [56]:
dic_renomear_colunas = {'Número':'NUMERO',
                        'Nome':'NOME',
                        'Data':'DATA',
                        'Nome':'CONTRIBUINTE',
                        'Empenhado(R$)':'VALOR',
                        'Liquidado(R$)':'LIQUIDADO',
                        'Pago(R$)': 'PAGO',
                        'Saldo(R$)': 'SALDO',
                        'Tipo':'TIPO',
                        'CODIGO DESPESA': 'COD_DESPESA',
                        'DESPESA':'TIPO_DESPESA',
                        'ORGAO':'CONTRIBUINTE'}

def renomear_colunas(df):
    df.rename(columns=dic_renomear_colunas, inplace=True)
    
for df in list_data_frames:
    renomear_colunas(df)

### Merge dos DF

In [57]:
dados_2020 = ['DATA','CPF/CNPJ','CONTRIBUINTE','VALOR','LIQUIDADO','PAGO','SALDO','TIPO']
dados_2022 = ['DATA','CONTRIBUINTE','VALOR','LIQUIDADO','PAGO','SALDO','TIPO']

df_dados_completos_2020 = pd.DataFrame(columns=dados_2020)
df_dados_completos_2022 = pd.DataFrame(columns=dados_2022)

In [58]:
def merge_dataframes(lista_dataframes, colunas_sem_valores):
    
    df_final = colunas_sem_valores.copy() # cria uma cópia do DataFrame das colunas sem valores
    
    for df in lista_dataframes:
        df_final = pd.merge(df_final, df, on=list(df_final.columns), how='outer') # mescla o DataFrame atual com o DataFrame das colunas
        
    return df_final

In [59]:
df_dados_completos_2020 = merge_dataframes(list_data_frames[:8],df_dados_completos_2020)

df_dados_completos_2020.drop('NUMERO', axis=1, inplace=True)
df_dados_completos_2020

,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,03/01/2013,CAGEPA-COM DE AGUA E ESGOTOS DA PARAIBA,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,30/01/2013,Prefeitura Municipal de Cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1
...,...,...,...,...,...,...,...,...
67869,31/12/2020,CAMARA MUNICIPAL DE CAJAZEIRAS,08.841.553/0001-89,64849.34,64849.34,64849.34,0.0,4
67870,31/12/2020,CAMARA MUNICIPAL DE CAJAZEIRAS,08.841.553/0001-89,11300.00,11300.00,11300.00,0.0,4
67871,31/12/2020,CAMARA MUNICIPAL DE CAJAZEIRAS,08.841.553/0001-89,52999.00,52999.00,52999.00,0.0,4
67872,31/12/2020,CAMARA MUNICIPAL DE CAJAZEIRAS,08.841.553/0001-89,137320.72,137320.72,137320.72,0.0,4


In [60]:
df_dados_completos_2022 = merge_dataframes(list_data_frames[8:],df_dados_completos_2022)

df_dados_completos_2022.drop('CODIGO ORGAO', axis=1, inplace=True)

df_dados_completos_2022

,DATA,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,31/12/2021,Fundo Municipal de Assistência Social,339039,Outros Serviços de Terceiros - Pessoa Jurídica,12/01/2022,10.45,10.45,10.45,0.00,1
1,31/12/2021,Fundo Municipal de Assistência Social,319004,Contratação por Tempo Determinado,11/01/2022,17820.00,17820.00,17820.00,0.00,1
2,31/12/2021,Fundo Municipal de Assistência Social,319004,Contratação por Tempo Determinado,11/01/2022,24525.03,24525.03,24525.03,0.00,1
3,31/12/2021,Fundo Municipal de Assistência Social,319011,Vencimentos e Vantagens Fixas - Pessoal Civil,11/01/2022,61859.51,61859.51,61859.51,0.00,1
4,31/12/2021,Secretaria Municipal de Desenvolvimento Humano,319011,Vencimentos e Vantagens Fixas - Pessoal Civil,11/01/2022,6500.00,6500.00,6500.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...
20437,17/01/2022,Câmara Municipal,319011,Vencimentos e Vantagens Fixas - Pessoal Civil,24/01/2022,31400.00,31400.00,31400.00,0.00,4
20438,17/01/2022,Câmara Municipal,319011,Vencimentos e Vantagens Fixas - Pessoal Civil,24/01/2022,29228.80,29228.80,29228.80,0.00,4
20439,17/01/2022,Câmara Municipal,319011,Vencimentos e Vantagens Fixas - Pessoal Civil,24/01/2022,154920.00,154920.00,154920.00,0.00,4
20440,03/01/2022,Câmara Municipal,339039,Outros Serviços de Terceiros - Pessoa Jurídica,20/01/2022,4000.00,3360.24,3360.24,474.76,4


### Tratamento coluna contribuintes

#### Primeira visualização dos dados

In [61]:
def tratar_coluna(df, coluna):
    
    # remove sinais de pontuação e símbolos
    table = str.maketrans('', '', string.punctuation)
    df[coluna] = df[coluna].apply(lambda x: str(x).translate(table))
    
    # remove acentuação gráfica
    df[coluna] = df[coluna].apply(lambda x: unidecode(x))
    
    # converte todos os caracteres para minúsculas
    df[coluna] = df[coluna].str.lower()
    
    return df


In [62]:
df_dados_completos_tratados_2020 = tratar_coluna(df_dados_completos_2020, 'CONTRIBUINTE')
df_dados_completos_tratados_2020

,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1
...,...,...,...,...,...,...,...,...
67869,31/12/2020,camara municipal de cajazeiras,08.841.553/0001-89,64849.34,64849.34,64849.34,0.0,4
67870,31/12/2020,camara municipal de cajazeiras,08.841.553/0001-89,11300.00,11300.00,11300.00,0.0,4
67871,31/12/2020,camara municipal de cajazeiras,08.841.553/0001-89,52999.00,52999.00,52999.00,0.0,4
67872,31/12/2020,camara municipal de cajazeiras,08.841.553/0001-89,137320.72,137320.72,137320.72,0.0,4


In [63]:
df_dados_completos_tratados_2022 = tratar_coluna(df_dados_completos_2022, 'CONTRIBUINTE')
df_dados_completos_tratados_2022 = tratar_coluna(df_dados_completos_2022, 'TIPO_DESPESA')


df_dados_completos_tratados_2022

,DATA,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,31/12/2021,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.00,1
1,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.00,1
2,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.00,1
3,31/12/2021,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.00,1
4,31/12/2021,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...
20437,17/01/2022,camara municipal,319011,vencimentos e vantagens fixas pessoal civil,24/01/2022,31400.00,31400.00,31400.00,0.00,4
20438,17/01/2022,camara municipal,319011,vencimentos e vantagens fixas pessoal civil,24/01/2022,29228.80,29228.80,29228.80,0.00,4
20439,17/01/2022,camara municipal,319011,vencimentos e vantagens fixas pessoal civil,24/01/2022,154920.00,154920.00,154920.00,0.00,4
20440,03/01/2022,camara municipal,339039,outros servicos de terceiros pessoa juridica,20/01/2022,4000.00,3360.24,3360.24,474.76,4


In [64]:
def filter_dataframe(df: pd.DataFrame, group_col: str, filter_crit: str) -> pd.DataFrame:
    """
    Filtra um DataFrame do Pandas de acordo com um critério definido em uma coluna específica e retorna um novo DataFrame
    sem perda de dados.

    Args:
        df (pd.DataFrame): DataFrame a ser filtrado.
        group_col (str): Nome da coluna do DataFrame que será utilizada para agrupamento.
        filter_crit (str): Critério utilizado para filtro dos dados.

    Returns:
        pd.DataFrame: DataFrame filtrado sem perda de dados.
    """
    # Cria um objeto com os grupos baseados na coluna 'group_col'
    groups = df.groupby(group_col)

    # Cria uma lista vazia para armazenar os DataFrames filtrados
    filtered_dfs = []

    # Loop que percorre cada grupo do objeto 'groups'
    for group_name, group_df in groups:
        # Filtra o DataFrame para manter apenas a primeira ocorrência de cada valor único na coluna parâmetro
        filtered_df = group_df.drop_duplicates(subset=[filter_crit], keep='first')
        # Adiciona o DataFrame filtrado à lista de DataFrames filtrados
        filtered_dfs.append(filtered_df)

    # Concatena os DataFrames filtrados em um novo DataFrame
    filtered_df = pd.concat(filtered_dfs)

    # Retorna o novo DataFrame sem perda de dados
    return filtered_df

In [65]:
df_contribuintes_receitas_2020 = pd.DataFrame(df_dados_completos_tratados_2020[['CPF/CNPJ','CONTRIBUINTE','TIPO']])
df_contribuintes_receitas_2020

,CPF/CNPJ,CONTRIBUINTE,TIPO
0,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,1
1,08.923.971/0001-15,prefeitura municipal de cajazeiras,1
2,00.000.000/0099-03,banco do brasil sa,1
3,00.000.000/0099-03,banco do brasil sa,1
4,00.000.000/0099-03,banco do brasil sa,1
...,...,...,...
67869,08.841.553/0001-89,camara municipal de cajazeiras,4
67870,08.841.553/0001-89,camara municipal de cajazeiras,4
67871,08.841.553/0001-89,camara municipal de cajazeiras,4
67872,08.841.553/0001-89,camara municipal de cajazeiras,4


In [66]:
df_contribuintes_receitas_2020 = filter_dataframe(df_contribuintes_receitas_2020, 'TIPO', 'CPF/CNPJ')
df_contribuintes_receitas_2020

,CPF/CNPJ,CONTRIBUINTE,TIPO
0,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,1
1,08.923.971/0001-15,prefeitura municipal de cajazeiras,1
2,00.000.000/0099-03,banco do brasil sa,1
19,09.283.185/0001-63,poder judiciario do estado da paraiba,1
34,738.558.394-68,henrique sergio alves da cunha,1
...,...,...,...
67797,41.207.317/0001-80,juvana moveis ltda,3
7609,08.841.553/0001-89,camara municipal de cajazeiras,4
7610,646.603.624-34,marcos helder,4
51150,08.923.971/0001-15,prefeitura municipal de cajazeiras,4


In [67]:
df_contribuintes_receitas_2022 = pd.DataFrame(df_dados_completos_tratados_2022[['CONTRIBUINTE', 'TIPO']])
df_contribuintes_receitas_2022

,CONTRIBUINTE,TIPO
0,fundo municipal de assistencia social,1
1,fundo municipal de assistencia social,1
2,fundo municipal de assistencia social,1
3,fundo municipal de assistencia social,1
4,secretaria municipal de desenvolvimento humano,1
...,...,...
20437,camara municipal,4
20438,camara municipal,4
20439,camara municipal,4
20440,camara municipal,4


In [68]:
df_contribuintes_receitas_2022 = filter_dataframe(df_contribuintes_receitas_2022,'TIPO','CONTRIBUINTE')
df_contribuintes_receitas_2022

,CONTRIBUINTE,TIPO
0,fundo municipal de assistencia social,1
4,secretaria municipal de desenvolvimento humano,1
13,fundo municipal de saude,1
74,secretaria municipal de educacao,1
77,secretaria municipal de juventude esporte e t...,1
79,procuradoria geral do municipio,1
82,secretaria municipal da fazenda publica,1
83,secretaria municipal de saude,1
86,secretaria de governo e articulacao politica,1
87,secretaria municipal de politicas publicas,1


#### Extraindo e enxugando as colunas contribuintes presentes nos dados das receitas dos DF de até 2020

In [69]:
def busca_df(df, coluna, busca):
    # Cria uma lista vazia para armazenar os valores correspondentes
    valores_correspondentes = []
    
    # Para cada valor na coluna especificada, verifica se a busca está presente no valor
    for valor in df[coluna]:
        if busca in str(valor):
            valores_correspondentes.append(valor)
    
    return valores_correspondentes

lista_string_municipal_2020 = busca_df(df_contribuintes_receitas_2020, 'CONTRIBUINTE', 'municipal')
lista_string_secretaria_2020 = busca_df(df_contribuintes_receitas_2020, 'CONTRIBUINTE', 'secretaria')
lista_string_sctrans_2020 = busca_df(df_contribuintes_receitas_2020, 'CONTRIBUINTE', 'sctrans')
lista_string_superintendencia_2020 = busca_df(df_contribuintes_receitas_2020, 'CONTRIBUINTE', 'superintendencia')
 


lista_string_municipal_2022 = busca_df(df_contribuintes_receitas_2022, 'CONTRIBUINTE', 'municipal')
lista_string_secretaria_2022 = busca_df(df_contribuintes_receitas_2022, 'CONTRIBUINTE', 'secretaria')
lista_string_sctrans_2022 = busca_df(df_contribuintes_receitas_2022, 'CONTRIBUINTE', 'sctrans')
lista_string_superintendencia_2022 = busca_df(df_contribuintes_receitas_2022, 'CONTRIBUINTE', 'superintendencia')



print(f'2020 - String municipal: {lista_string_municipal_2020}')
print(f'2020 - String secretaria: {lista_string_secretaria_2020}')
print(f'2020 - String SCTRANS: : {lista_string_sctrans_2020}')
print(f'2020 - String superintendencia: : {lista_string_superintendencia_2022}')


print(f'2022 - String municipal: {lista_string_municipal_2022}')
print(f'2022 - String secretaria: {lista_string_secretaria_2022}')
print(f'2022 - String SCTRANS: : {lista_string_sctrans_2022}')
print(f'2022 - String superintendencia: {lista_string_superintendencia_2022}')

2020 - String municipal: ['prefeitura municipal de cajazeiras', 'fundo municipal de incentivo a cultura  fuminc', 'fundo municipal de saude', 'ame saude  consorcio intermunicipal de saude dos municipios do alto sertao para', 'fundo municipal de assistencia social de cajazeiras', 'cf consultoria tributaria municipal eireli', 'prefeitura municipal de aurora', 'fundo municipal de assistencia social', 'ipam  inst prev assist municipal', 'prefeitura municipal de cajazeiras', 'municipio de cajazeiras fundo municipal de saude  fms', 'ipam  instituto de previdencia e assistencia municipal de cajazeiras', 'prefeitura municipal de cajazeiras', 'camara municipal de cajazeiras', 'prefeitura municipal de cajazeiras', 'prefeitura municipal de cajazeiras']
2020 - String secretaria: ['secretaria de estado da receita', 'secretaria do tesouro nacional', 'secretaria de estado do planejamento e gestao', 'secretaria da receita federal do brasil', 'secretaria de educacao e cultura', 'conselho de secretarias

In [70]:
dic_padronizar_valores = {' superintendencia cajazeirense de transito':'superintendencia cajazeirense de transito',
                          'fundo municipal de saude':'fundo municipal de saude',
                          ' fundo municipal de saude':'fundo municipal de saude',
                          ' instituto de previdencia e assistencia municipal de cajazeiras':'instituto de previdencia e assistencia municipal de cajazeiras',
                          ' camara municipal':'camara municipal de cajazeiras',
                          'ipam  instituto de previdencia e assistencia municipal de cajazeiras':'instituto de previdencia e assistencia municipal de cajazeiras',
                          'municipio de cajazeiras fundo municipal de saude  fms':'fundo municipal de saude',
                          'ipam  inst prev assist municipal':'instituto de previdencia e assistencia municipal de cajazeiras'}

In [71]:
def altera_df(df, dicionario, coluna):
    # Cria um novo DataFrame para evitar perda de dados do antigo
    novo_df = df.copy()
    
    # Para cada chave no dicionário, encontra as linhas do DataFrame em que o valor na coluna especificada corresponde à chave
    for chave in dicionario:
        novo_df.loc[novo_df[coluna] == chave, coluna] = dicionario[chave]
        
    return novo_df

df_contribuintes_receitas_2020 = altera_df(df_contribuintes_receitas_2020, dic_padronizar_valores, 'CONTRIBUINTE')
df_contribuintes_receitas_2020

,CPF/CNPJ,CONTRIBUINTE,TIPO
0,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,1
1,08.923.971/0001-15,prefeitura municipal de cajazeiras,1
2,00.000.000/0099-03,banco do brasil sa,1
19,09.283.185/0001-63,poder judiciario do estado da paraiba,1
34,738.558.394-68,henrique sergio alves da cunha,1
...,...,...,...
67797,41.207.317/0001-80,juvana moveis ltda,3
7609,08.841.553/0001-89,camara municipal de cajazeiras,4
7610,646.603.624-34,marcos helder,4
51150,08.923.971/0001-15,prefeitura municipal de cajazeiras,4


In [72]:
df_contribuintes_receitas_2022 = altera_df(df_contribuintes_receitas_2022, dic_padronizar_valores, 'CONTRIBUINTE')
df_contribuintes_receitas_2022

,CONTRIBUINTE,TIPO
0,fundo municipal de assistencia social,1
4,secretaria municipal de desenvolvimento humano,1
13,fundo municipal de saude,1
74,secretaria municipal de educacao,1
77,secretaria municipal de juventude esporte e t...,1
79,procuradoria geral do municipio,1
82,secretaria municipal da fazenda publica,1
83,secretaria municipal de saude,1
86,secretaria de governo e articulacao politica,1
87,secretaria municipal de politicas publicas,1


### Merge dos DF Contribuintes

In [73]:
def merge_dataframes(df1, df2):
    
    # Cria cópias dos dataframes para evitar perda de dados
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    
    # Itera sobre cada linha do primeiro dataframe
    for i, row1 in df1_copy.iterrows():
        
        # Itera sobre cada linha do segundo dataframe
        for j, row2 in df2_copy.iterrows():
            
            # Compara o valor da coluna 'CONTRIBUINTE' em ambas as linhas
            if row1['CONTRIBUINTE'] == row2['CONTRIBUINTE']:
                
                # Atribui o valor da coluna 'CPF/CNPJ' do primeiro dataframe no segundo
                df2_copy.at[j, 'CPF/CNPJ'] = row1['CPF/CNPJ']
    
    # Faz o merge dos dois dataframes sem valores duplicados
    merged_df = pd.concat([df1_copy, df2_copy]).reset_index(drop=True)
    
    return merged_df

In [74]:
df_merge_contribuintes = merge_dataframes(df_contribuintes_receitas_2020, df_contribuintes_receitas_2022)
df_merge_contribuintes

,CPF/CNPJ,CONTRIBUINTE,TIPO
0,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,1
1,08.923.971/0001-15,prefeitura municipal de cajazeiras,1
2,00.000.000/0099-03,banco do brasil sa,1
3,09.283.185/0001-63,poder judiciario do estado da paraiba,1
4,738.558.394-68,henrique sergio alves da cunha,1
...,...,...,...
6307,NaN,secretaria municipal de juventude e esporte,1
6308,NaN,secretaria municipal do desenvolvimento rural...,1
6309,12.724.464/0001-20,instituto de previdencia e assistencia municip...,2
6310,04.342.393/0001-72,superintendencia cajazeirense de transito,3


In [75]:
def gerar_codigo_unico(df):
    # Verificar se a coluna "CODIGO" já existe no dataframe
    if "CODIGO" in df.columns:
        # Criar uma lista de espécies únicas presentes na coluna TIPO do dataframe
        especies = df['TIPO'].unique()
        # Gerar um código aleatório de 4 dígitos para cada espécie única
        codigos = [ ''.join([str(np.random.randint(0, 9)) for _ in range(4)]) for _ in range(len(especies)) ]
        # Criar um dicionário que mapeia cada espécie única a seu respectivo código gerado
        especie_codigo_dict = dict(zip(especies, codigos))
        # Atualizar a coluna "CODIGO" do dataframe com os novos códigos gerados para cada espécie única
        df["CODIGO"] = df["TIPO"].map(especie_codigo_dict)
        return df
    
    # Criar uma cópia do dataframe original para evitar perdas de dados
    df_novo = df.copy()
    
    # Criar uma lista de espécies únicas presentes na coluna TIPO do dataframe
    especies = df_novo['TIPO'].unique()
    
    # Gerar um código aleatório de 4 dígitos para cada espécie única e adicionar o código gerado a uma lista
    codigos = [ ''.join([str(np.random.randint(0, 9)) for _ in range(4)]) for _ in range(len(especies)) ]
    
    # Criar um dicionário que mapeia cada espécie única a seu respectivo código gerado
    especie_codigo_dict = dict(zip(especies, codigos))
    
    # Adicionar uma coluna "CODIGO" ao dataframe com os códigos gerados para cada espécie única
    df_novo.insert(0, "CODIGO", df_novo["TIPO"].map(especie_codigo_dict))
    
    return df_novo

def preencher_cpf_cnpj(df):
    # Cria uma cópia do dataframe para evitar a perda dos dados originais
    df_novo = df.copy()

    # Verifica se existem células vazias na coluna 'CPF/CNPJ'
    cnpj_vazio = df_novo['CPF/CNPJ'].isnull() | (df_novo['CPF/CNPJ'] == '')

    # Cria um array com valores únicos de CPF/CNPJ para evitar a criação de códigos repetidos
    cpf_cnpj_unicos = df_novo['CPF/CNPJ'].unique()

    # Itera pelas linhas do dataframe com CPF/CNPJ vazio
    for i, row in df_novo[cnpj_vazio].iterrows():

        # Salva o código do tipo para anexar ao código final
        codigo_tipo = df_novo.loc[i,'CODIGO']

        # Gera um novo código único de até 14 dígitos
        novo_codigo = np.random.randint(10**10, 10**14, dtype=np.int64)

        # Formata o código gerado para inserir o ponto após os quatro primeiros dígitos
        novo_cpf_cnpj = f"{str(codigo_tipo)[:4]}.{str(novo_codigo)[4:]}"

        # Verifica se o código gerado já existe no dataframe original
        while novo_cpf_cnpj in cpf_cnpj_unicos:
            novo_codigo = np.random.randint(10**10, 10**14, dtype=np.int64)
            novo_cpf_cnpj = f"{str(codigo_tipo)[:4]}.{str(novo_codigo)[4:]}"

        # Adiciona o novo código na célula vazia correspondente
        df_novo.loc[i, 'CPF/CNPJ'] = novo_cpf_cnpj

        # Adiciona o novo código na lista de CPF/CNPJ únicos
        cpf_cnpj_unicos = np.append(cpf_cnpj_unicos, novo_cpf_cnpj)
        
    return df_novo

In [76]:
df_merge_contribuintes.loc[0,'CPF/CNPJ']

'09.123.654/0001-87'

In [77]:
df_merge_contribuintes = gerar_codigo_unico(df_merge_contribuintes)
df_merge_contribuintes = preencher_cpf_cnpj(df_merge_contribuintes)
df_merge_contribuintes

,CODIGO,CPF/CNPJ,CONTRIBUINTE,TIPO
0,4163,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,1
1,4163,08.923.971/0001-15,prefeitura municipal de cajazeiras,1
2,4163,00.000.000/0099-03,banco do brasil sa,1
3,4163,09.283.185/0001-63,poder judiciario do estado da paraiba,1
4,4163,738.558.394-68,henrique sergio alves da cunha,1
...,...,...,...,...
6307,4163,4163.0383613688,secretaria municipal de juventude e esporte,1
6308,4163,4163.7718374060,secretaria municipal do desenvolvimento rural...,1
6309,2177,12.724.464/0001-20,instituto de previdencia e assistencia municip...,2
6310,5216,04.342.393/0001-72,superintendencia cajazeirense de transito,3


In [78]:
df_merge_contribuintes.to_csv('dados_contribuintes.csv', index=False, sep=';')

### Tratamento da coluna data - Dimensão Tempo

#### Função para mergear os df pela coluna DATA

In [79]:
def merge_columns(dataframes_list, column_name):
    """
    Esta função recebe uma lista de pandas dataframes e o nome de uma coluna que
    deve ser usada para fazer a junção dos dataframes. Retorna um novo dataframe contendo
    apenas os valores únicos da coluna especificada dos dataframes originais.

    Argumentos:
    - dataframes_list: uma lista de pandas dataframes que serão juntados
    - column_name: o nome da coluna que deve ser usada para a junção

    Retorna:
    - um novo pandas dataframe contendo apenas os valores únicos da coluna especificada
    dos dataframes originais
    """
    # Verifica se a lista de dataframes é válida
    if not isinstance(dataframes_list, list) or not all(isinstance(df, pd.DataFrame) for df in dataframes_list):
        raise ValueError("A entrada deve ser uma lista de pandas dataframes")

    # Verifica se o nome da coluna é válido
    if not all(column_name in df.columns for df in dataframes_list):
        raise ValueError("O nome da coluna especificada não está presente em todos os dataframes")

    # Concatena os valores da coluna especificada de todos os dataframes
    column_values = pd.concat([df[column_name] for df in dataframes_list], ignore_index=True)

    # Remove valores duplicados da coluna concatenada
    unique_values = column_values.drop_duplicates()

    # Cria um novo dataframe com os valores únicos da coluna
    merged_df = pd.DataFrame({column_name: unique_values})

    return merged_df

In [80]:
df_datas_mergeado = merge_columns(list_data_frames, 'DATA')
df_datas_mergeado

,DATA
0,03/01/2013
1,30/01/2013
12,05/02/2013
13,07/02/2013
14,08/02/2013
...,...
86812,21/01/2022
86946,22/08/2022
87096,22/01/2022
87216,03/11/2022


In [81]:
def converte_para_datetime(df, nome_coluna):
    novo_df = df.copy() # Criando uma cópia do dataframe para evitar perda de dados
    novo_df[nome_coluna] = pd.to_datetime(novo_df[nome_coluna], format='%d/%m/%Y') # Convertendo a coluna para datetime com o formato especificado
    novo_df['ANO'] = pd.to_datetime(novo_df[nome_coluna]).dt.year # Extrai o ano da coluna "DATA" e adiciona como nova coluna "ANO"
    novo_df['MES'] = pd.to_datetime(novo_df[nome_coluna]).dt.month # Extrai o mês da coluna "DATA" e adiciona como nova coluna "MES"
    novo_df['DIA'] = pd.to_datetime(novo_df[nome_coluna]).dt.day # Extrai o dia da coluna "DATA" e adiciona como nova coluna "DIA"
    return novo_df

In [82]:
df_datas_tratadas = converte_para_datetime(df_datas_mergeado, 'DATA')
df_datas_tratadas

,DATA,ANO,MES,DIA
0,2013-01-03,2013,1,3
1,2013-01-30,2013,1,30
12,2013-02-05,2013,2,5
13,2013-02-07,2013,2,7
14,2013-02-08,2013,2,8
...,...,...,...,...
86812,2022-01-21,2022,1,21
86946,2022-08-22,2022,8,22
87096,2022-01-22,2022,1,22
87216,2022-11-03,2022,11,3


In [83]:
meses_do_ano = {1: 'janeiro', 
                2: 'fevereiro', 
                3: 'março', 
                4: 'abril', 
                5: 'maio', 
                6: 'junho', 
                7: 'julho', 
                8: 'agosto', 
                9: 'setembro', 
                10: 'outubro', 
                11: 'novembro', 
                12: 'dezembro'}

In [84]:
trimestres_ano = {'janeiro':'primeiro trimestre','fevereiro':'primeiro trimestre','março':'primeiro trimestre',
                  'abril':'segundo trimestre','maio':'segundo trimestre','junho':'segundo trimestre',
                  'julho':'terceiro trimestre','agosto':'terceiro trimestre','setembro':'terceiro trimestre',
                  'outubro':'quarto trimestre','novembro':'quarto trimestre','dezembro':'quarto trimestre',}
codigo_trimestre = {'primeiro trimestre': 1, 'segundo trimestre': 2, 'terceiro trimestre': 3, 'quarto trimestre':4}

In [85]:
semestres_ano = {'primeiro trimestre':'primeiro semestre','segundo trimestre':'primeiro semestre',
                 'terceiro trimestre':'segundo semestre','quarto trimestre':'segundo semestre'}
codigo_semestre = {'primeiro semestre': 1, 'segundo semestre': 2}

In [86]:
def alterar_colunas(df, coluna):
    # criar cópia do dataframe para não perder dados originais
    novo_df = df.copy()
    
    # usar o método replace() do pandas para trocar os valores da coluna pelo dicionário
    novo_df[coluna] = novo_df[coluna].replace(meses_do_ano)
    # criação da coluna TRIMESTRE com base na coluna MES e no dicionario de trimestres
    novo_df['TRIMESTRE'] = novo_df['MES'].map(trimestres_ano)
    # criação da coluna COD_TRIMESTRE com base na coluna TRIMESTRE e no dicionario de códigos de trimestres
    novo_df['COD_TRIMESTRE'] = novo_df['TRIMESTRE'].map(codigo_trimestre)
    # criação da coluna SEMESTRE com base na coluna TRIMESTRE e no dicionario de semestres
    novo_df['SEMESTRE'] = novo_df['TRIMESTRE'].map(semestres_ano)
    # criação da coluna COD_SEMESTRE com base na coluna SEMESTRE e no dicionario de códigos de trimestres
    novo_df['COD_SEMESTRE'] = novo_df['SEMESTRE'].map(codigo_semestre)

    return novo_df

In [87]:
df_datas_tratadas = alterar_colunas(df_datas_tratadas, 'MES')
df_datas_tratadas

,DATA,ANO,MES,DIA,TRIMESTRE,COD_TRIMESTRE,SEMESTRE,COD_SEMESTRE
0,2013-01-03,2013,janeiro,3,primeiro trimestre,1,primeiro semestre,1
1,2013-01-30,2013,janeiro,30,primeiro trimestre,1,primeiro semestre,1
12,2013-02-05,2013,fevereiro,5,primeiro trimestre,1,primeiro semestre,1
13,2013-02-07,2013,fevereiro,7,primeiro trimestre,1,primeiro semestre,1
14,2013-02-08,2013,fevereiro,8,primeiro trimestre,1,primeiro semestre,1
...,...,...,...,...,...,...,...,...
86812,2022-01-21,2022,janeiro,21,primeiro trimestre,1,primeiro semestre,1
86946,2022-08-22,2022,agosto,22,terceiro trimestre,3,segundo semestre,2
87096,2022-01-22,2022,janeiro,22,primeiro trimestre,1,primeiro semestre,1
87216,2022-11-03,2022,novembro,3,quarto trimestre,4,segundo semestre,2


In [88]:
df_datas_tratadas.to_csv('dados_tempo.csv', index=False, sep=';')

### Tratamento das classes das despesas

In [89]:
df_dados_completos_tratados_2022.head()

,DATA,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,31/12/2021,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1
1,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1
2,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1
3,31/12/2021,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1
4,31/12/2021,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1


In [90]:
df_classes_despesas_2022 = df_dados_completos_tratados_2022[['COD_DESPESA','TIPO_DESPESA']]
df_classes_despesas_2022 = df_classes_despesas_2022.copy()
df_classes_despesas_2022.drop_duplicates(subset='COD_DESPESA', inplace=True)
df_classes_despesas_2022 = df_classes_despesas_2022.loc[:, ~df_classes_despesas_2022.columns.str.startswith('Unnamed')].reset_index(drop=True)
df_classes_despesas_2022

,COD_DESPESA,TIPO_DESPESA
0,339039,outros servicos de terceiros pessoa juridica
1,319004,contratacao por tempo determinado
2,319011,vencimentos e vantagens fixas pessoal civil
3,339030,material de consumo
4,335041,contribuicoes
5,339036,outros servicos de terceiros pessoa fisica
6,339091,sentencas judiciais
7,339048,outros auxilios financeiros a pessoas fisicas
8,469071,principal da divida contratual resgatado
9,319113,contribuicoes patronais


In [91]:
df_classes_despesas_2022.to_csv('classes_despesas_2021_2022.csv', index=False ,sep=';')

### Tratamento dos tipos de despesas

In [92]:
df_dados_completos_tratados_2020.head()

,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


In [93]:
df_dados_completos_tratados_2022.head()

,DATA,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,31/12/2021,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1
1,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1
2,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1
3,31/12/2021,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1
4,31/12/2021,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1


In [94]:
def acrescenta_codigo_dados_2021_2022(df1, df2):
    # Renomeando a coluna 'CPF/CNPJ' para 'CPF_CNPJ'
    df2 = df2.rename(columns={'CPF/CNPJ': 'CPF_CNPJ'})

    # Usando merge para combinar os dataframes
    df_merged = df1.merge(df2, on=['CONTRIBUINTE', 'TIPO'], how='left')

    return df_merged

In [95]:
df_dados_completos_tratados_2022['CONTRIBUINTE'].replace(dic_padronizar_valores, inplace=True)
df_dados_completos_tratados_2022 = acrescenta_codigo_dados_2021_2022(df_dados_completos_tratados_2022, df_merge_contribuintes)
df_dados_completos_tratados_2022

,DATA,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CODIGO,CPF_CNPJ
0,31/12/2021,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.00,1,4163,4163.247489777
1,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.00,1,4163,4163.247489777
2,31/12/2021,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.00,1,4163,4163.247489777
3,31/12/2021,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.00,1,4163,4163.247489777
4,31/12/2021,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.00,1,4163,4163.5179847145
...,...,...,...,...,...,...,...,...,...,...,...,...
27498,17/01/2022,camara municipal de cajazeiras,319011,vencimentos e vantagens fixas pessoal civil,24/01/2022,154920.00,154920.00,154920.00,0.00,4,7507,08.841.553/0001-89
27499,03/01/2022,camara municipal de cajazeiras,339039,outros servicos de terceiros pessoa juridica,20/01/2022,4000.00,3360.24,3360.24,474.76,4,7507,08.841.553/0001-89
27500,03/01/2022,camara municipal de cajazeiras,339039,outros servicos de terceiros pessoa juridica,20/01/2022,4000.00,3360.24,3360.24,474.76,4,7507,08.841.553/0001-89
27501,03/01/2022,camara municipal de cajazeiras,339039,outros servicos de terceiros pessoa juridica,20/01/2022,4000.00,2198.75,2198.75,310.20,4,7507,08.841.553/0001-89


In [96]:
df_dados_completos_tratados_2022.drop(['TIPO_DESPESA','CODIGO','CONTRIBUINTE','DATA INCLUSAO'], axis=1, inplace=True)
df_dados_completos_tratados_2022.rename({'COD_DESPESA':'COD_DESPESA_INTERNO','TIPO':'FONTE_DADOS'}, axis=1, inplace=True)
df_dados_completos_tratados_2022 = df_dados_completos_tratados_2022.reindex(['COD_DESPESA_INTERNO','CPF_CNPJ','DATA','VALOR','LIQUIDADO','PAGO','SALDO','FONTE_DADOS'], axis=1)
df_dados_completos_tratados_2022['DATA'] = pd.to_datetime(df_dados_completos_tratados_2022['DATA'], format='%d/%m/%Y')

df_dados_completos_tratados_2022.head()

,COD_DESPESA_INTERNO,CPF_CNPJ,DATA,VALOR,LIQUIDADO,PAGO,SALDO,FONTE_DADOS
0,339039,4163.247489777,2021-12-31,10.45,10.45,10.45,0.0,1
1,319004,4163.247489777,2021-12-31,17820.00,17820.00,17820.00,0.0,1
2,319004,4163.247489777,2021-12-31,24525.03,24525.03,24525.03,0.0,1
3,319011,4163.247489777,2021-12-31,61859.51,61859.51,61859.51,0.0,1
4,319011,4163.5179847145,2021-12-31,6500.00,6500.00,6500.00,0.0,1


In [97]:
df_dados_completos_tratados_2020.head()

,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


In [98]:
df_dados_completos_tratados_2020.drop(['CONTRIBUINTE'], axis=1, inplace=True)
df_dados_completos_tratados_2020.rename({'TIPO':'FONTE_DADOS'}, axis=1, inplace=True)
df_dados_completos_tratados_2020 = df_dados_completos_tratados_2020.reindex(['CPF_CNPJ','DATA','VALOR','LIQUIDADO','PAGO','SALDO','FONTE_DADOS'], axis=1)
df_dados_completos_tratados_2020['DATA'] = pd.to_datetime(df_dados_completos_tratados_2020['DATA'], format='%d/%m/%Y')

df_dados_completos_tratados_2020.head()

,CPF_CNPJ,DATA,VALOR,LIQUIDADO,PAGO,SALDO,FONTE_DADOS
0,NaN,2013-01-03,45.68,45.68,45.68,0.0,1
1,NaN,2013-01-30,13309.74,13309.74,13309.74,0.0,1
2,NaN,2013-01-30,37073.10,37073.10,37073.10,0.0,1
3,NaN,2013-01-30,3223.75,3223.75,3223.75,0.0,1
4,NaN,2013-01-30,39698.37,39698.37,39698.37,0.0,1


In [99]:
df_dados_completos_tratados_2020.to_csv('dados_receitas_2013_2020.csv', index=False, sep=';')
df_dados_completos_tratados_2022.to_csv('dados_receitas_2021_2022.csv', index=False, sep=';')
